# Weekly returns for European funds (DuckDB)

## Context
We are given daily fund performance exported from a performance system and a calendar describing which days are open/closed.
Business users need a dataset that includes **daily returns** and **weekly returns** for **European markets** (open 5 days/week).

## Weekly return formula
Weekly return is computed by compounding daily returns for the trading days in a week:

\[ R = \prod_{i=1}^{n} (1 + r_i) - 1 \]


In [2]:
# If running locally and duckdb is not installed, uncomment:
# !pip install duckdb pandas

import duckdb
import pandas as pd
from pathlib import Path


In [3]:
# Paths (assumes notebook is located in resources/)
BASE_DIR = Path('.').resolve()
CALENDAR_CSV = BASE_DIR / 'calendar.csv'
DATA_CSV = BASE_DIR / 'data.csv'

CALENDAR_CSV, DATA_CSV


(PosixPath('/Users/soniaresende/Documents/Github/develop/Pictet/global_footprint_network_use_case/resources/calendar.csv'),
 PosixPath('/Users/soniaresende/Documents/Github/develop/Pictet/global_footprint_network_use_case/resources/data.csv'))

## Solution (step 1): Read CSV files and load into DuckDB

In [4]:
con = duckdb.connect(database=':memory:')

con.execute("""
CREATE OR REPLACE TABLE calendar AS
SELECT
  CAST(date AS DATE) AS date,
  CAST(day_of_week AS INTEGER) AS day_of_week,
  text_day_of_week,
  text_month_of_year,
  CAST(is_holiday AS BOOLEAN) AS is_holiday,
  CAST(is_working_day AS BOOLEAN) AS is_working_day
FROM read_csv_auto(?, header=true)
""", [str(CALENDAR_CSV)])

con.execute("""
CREATE OR REPLACE TABLE perf_daily AS
SELECT
  fund_code,
  CAST(market_date AS DATE) AS market_date,
  retunr_type AS return_type,
  CAST(daily_return AS DOUBLE) AS daily_return
FROM read_csv_auto(?, header=true)
""", [str(DATA_CSV)])

con.execute('SELECT COUNT(*) AS n_calendar_rows FROM calendar').df(), con.execute('SELECT COUNT(*) AS n_perf_rows FROM perf_daily').df()


(   n_calendar_rows
 0           146097,
    n_perf_rows
 0          472)

## Identify the business key
A daily performance record is uniquely identified by:
- `FUND_CODE`
- `MARKET_DATE`
- `RETURN_TYPE` (column name `RETUNR_TYPE` in the CSV)

We validate this by checking duplicates.

In [5]:
con.execute("""
SELECT
  fund_code,
  market_date,
  return_type,
  COUNT(*) AS cnt
FROM perf_daily
GROUP BY 1,2,3
HAVING COUNT(*) > 1
ORDER BY cnt DESC
""").df()


,FUND_CODE,market_date,return_type,cnt


## Solution (step 2): Compute weekly returns in SQL

### Trading days filter (Europe)
We filter to market open days using `calendar.csv`:
- `is_working_day = true` (Mon–Fri)
- `is_holiday = false` (exclude holidays)

### Weekly compounding
We implement the product in SQL as:
- `prod(1 + r)` = `exp(sum(ln(1 + r)))`
- `weekly_return` = `exp(sum(ln(1 + r))) - 1`

Null daily returns are ignored (missing values).

In [6]:
weekly_returns_df = con.execute("""
WITH market_open_days AS (
  SELECT
    p.fund_code,
    p.return_type,
    p.market_date,
    p.daily_return
  FROM perf_daily p
  INNER JOIN calendar c
    ON c.date = p.market_date
  WHERE c.is_working_day = TRUE
    AND c.is_holiday = FALSE
),
weekly_grouped AS (
  SELECT
    fund_code,
    return_type,
    date_trunc('week', market_date) AS week_start_date,
    COUNT(daily_return) AS n_observations,
    MIN(market_date) AS first_market_date_in_week,
    MAX(market_date) AS last_market_date_in_week,
    EXP(SUM(LN(1.0 + daily_return))) - 1.0 AS weekly_return
  FROM market_open_days
  WHERE daily_return IS NOT NULL
  GROUP BY 1,2,3
)
SELECT *
FROM weekly_grouped
ORDER BY fund_code, return_type, week_start_date
""").df()

weekly_returns_df


,FUND_CODE,return_type,week_start_date,n_observations,first_market_date_in_week,last_market_date_in_week,weekly_return
0,FUND_01,TYPE_A,2024-12-30,2,2025-01-02,2025-01-03,0.197955
1,FUND_01,TYPE_A,2025-01-06,5,2025-01-06,2025-01-10,0.986250
2,FUND_01,TYPE_A,2025-01-13,5,2025-01-13,2025-01-17,1.386146
3,FUND_01,TYPE_A,2025-01-20,5,2025-01-20,2025-01-24,1.323840
4,FUND_01,TYPE_A,2025-01-27,5,2025-01-27,2025-01-31,1.919325
...,...,...,...,...,...,...,...
67,FUND_04,TYPE_B,2025-01-27,5,2025-01-27,2025-01-31,2.479175
68,FUND_04,TYPE_B,2025-02-03,5,2025-02-03,2025-02-07,0.784843
69,FUND_04,TYPE_B,2025-02-10,5,2025-02-10,2025-02-14,1.874083
70,FUND_04,TYPE_B,2025-02-17,5,2025-02-17,2025-02-21,1.829315


## Expected result
The output above is the weekly return view your dashboard can consume (one row per `fund_code`, `return_type`, `week_start_date`).

## Explanation (brief)
1. Loaded `calendar.csv` and `data.csv` into DuckDB tables.
2. Identified the business key at daily grain: (`FUND_CODE`, `MARKET_DATE`, `RETURN_TYPE`).
3. Filtered to European open days using the calendar (`is_working_day=true` and `is_holiday=false`).
4. Computed weekly returns by compounding daily returns within each week using `EXP(SUM(LN(1+r))) - 1`.


## Data quality checks (recommended)

Before publishing a management-facing view, it is good practice to add a few lightweight data quality checks:

1. **Calendar coverage**: ensure every `MARKET_DATE` in the performance feed exists in `calendar.csv` (otherwise the `INNER JOIN` would silently drop rows).
2. **Valid values for compounding**: the weekly compounding uses `LN(1 + r)`, which requires `1 + r > 0` (i.e., `r > -1`). Values `<= -1` indicate data issues (or require special handling).

These checks should normally return **empty** (or near-empty) results.


In [7]:
# 1) Calendar coverage: performance dates missing in calendar
missing_calendar_dates_df = con.execute("""
SELECT
  p.market_date,
  COUNT(*) AS cnt
FROM perf_daily p
LEFT JOIN calendar c
  ON c.date = p.market_date
WHERE c.date IS NULL
GROUP BY 1
ORDER BY 1
""").df()

missing_calendar_dates_df


,market_date,cnt


In [8]:
# 2) Invalid returns for LN(1+r): daily_return <= -1
invalid_returns_df = con.execute("""
SELECT
  fund_code,
  market_date,
  return_type,
  daily_return
FROM perf_daily
WHERE daily_return IS NOT NULL
  AND daily_return <= -1
ORDER BY market_date, fund_code, return_type
""").df()

invalid_returns_df


,FUND_CODE,market_date,return_type,daily_return


## Weekly returns (strict completeness option)

By default, the weekly return calculation ignores null `DAILY_RETURN` values and compounds the available observations.

If the business requires **complete weeks only** (e.g., all trading days in that week must be present), we can:
- compute the expected number of trading days from the calendar for each week, and
- keep only weeks where `n_observations = expected_trading_days`.


In [9]:
weekly_returns_strict_df = con.execute("""
WITH expected_days AS (
  SELECT
    date_trunc('week', date) AS week_start_date,
    COUNT(*) AS expected_trading_days
  FROM calendar
  WHERE is_working_day = TRUE
    AND is_holiday = FALSE
  GROUP BY 1
),
market_open_days AS (
  SELECT
    p.fund_code,
    p.return_type,
    p.market_date,
    p.daily_return
  FROM perf_daily p
  INNER JOIN calendar c
    ON c.date = p.market_date
  WHERE c.is_working_day = TRUE
    AND c.is_holiday = FALSE
),
weekly_grouped AS (
  SELECT
    fund_code,
    return_type,
    date_trunc('week', market_date) AS week_start_date,
    COUNT(daily_return) AS n_observations,
    EXP(SUM(LN(1.0 + daily_return))) - 1.0 AS weekly_return
  FROM market_open_days
  WHERE daily_return IS NOT NULL
  GROUP BY 1,2,3
)
SELECT
  w.fund_code,
  w.return_type,
  w.week_start_date,
  e.expected_trading_days,
  w.n_observations,
  w.weekly_return
FROM weekly_grouped w
INNER JOIN expected_days e
  ON e.week_start_date = w.week_start_date
WHERE w.n_observations = e.expected_trading_days
ORDER BY w.fund_code, w.return_type, w.week_start_date
""").df()

weekly_returns_strict_df


,FUND_CODE,return_type,week_start_date,expected_trading_days,n_observations,weekly_return
0,FUND_01,TYPE_A,2025-01-06,5,5,0.986250
1,FUND_01,TYPE_A,2025-01-13,5,5,1.386146
2,FUND_01,TYPE_A,2025-01-20,5,5,1.323840
3,FUND_01,TYPE_A,2025-01-27,5,5,1.919325
4,FUND_01,TYPE_A,2025-02-03,5,5,2.525267
...,...,...,...,...,...,...
59,FUND_04,TYPE_B,2025-01-27,5,5,2.479175
60,FUND_04,TYPE_B,2025-02-03,5,5,0.784843
61,FUND_04,TYPE_B,2025-02-10,5,5,1.874083
62,FUND_04,TYPE_B,2025-02-17,5,5,1.829315


## Assumptions

- **Business key (daily grain)**: (`FUND_CODE`, `MARKET_DATE`, `RETURN_TYPE`).
- **Trading days (Europe)**: `is_working_day = true` and `is_holiday = false` (holiday overrides working day).
- **Week definition**: `date_trunc('week', market_date)` (week starts on Monday).
- **Missing daily returns**: `NULL` values are treated as *missing observations* (not 0%).
- **Compounding validity**: weekly compounding uses `LN(1 + r)` so it requires `r > -1`.

In a real production implementation, these assumptions should be confirmed with business stakeholders and documented in the data contract.


## Calendar data quality checks

Because the calendar drives the definition of *trading days*, it is important to validate it.
The checks below help detect common issues:

- Duplicate dates in the calendar.
- Dates marked both as holiday and working day (inconsistent flags).
- Weekend days marked as working days.


In [ ]:
# 1) Calendar duplicate dates (should be empty)
calendar_duplicate_dates_df = con.execute("""
SELECT
  date,
  COUNT(*) AS cnt
FROM calendar
GROUP BY 1
HAVING COUNT(*) > 1
ORDER BY date
""").df()

calendar_duplicate_dates_df


In [ ]:
# 2) Inconsistent flags: holiday and working day at the same time
calendar_holiday_working_conflicts_df = con.execute("""
SELECT
  date,
  day_of_week,
  text_day_of_week,
  is_holiday,
  is_working_day
FROM calendar
WHERE is_holiday = TRUE
  AND is_working_day = TRUE
ORDER BY date
""").df()

calendar_holiday_working_conflicts_df


In [ ]:
# 3) Weekend marked as working day
calendar_weekend_working_df = con.execute("""
SELECT
  date,
  day_of_week,
  text_day_of_week,
  is_working_day
FROM calendar
WHERE day_of_week IN (6, 7)
  AND is_working_day = TRUE
ORDER BY date
""").df()

calendar_weekend_working_df


## Weekly returns (with completeness KPI)

In addition to weekly returns, it is useful to publish transparency metrics for BI consumers:

- `expected_trading_days`: number of trading days in that week (from calendar)
- `observed_trading_days`: number of non-null returns observed
- `coverage_ratio`: observed / expected
- `is_complete_week`: whether the week is fully populated

This allows dashboard users to spot partial weeks (e.g., due to missing data or market holidays) without hiding them.


In [ ]:
weekly_returns_with_coverage_df = con.execute("""
WITH expected_days AS (
  SELECT
    date_trunc('week', date) AS week_start_date,
    COUNT(*) AS expected_trading_days
  FROM calendar
  WHERE is_working_day = TRUE
    AND is_holiday = FALSE
  GROUP BY 1
),
market_open_days AS (
  SELECT
    p.fund_code,
    p.return_type,
    p.market_date,
    p.daily_return
  FROM perf_daily p
  INNER JOIN calendar c
    ON c.date = p.market_date
  WHERE c.is_working_day = TRUE
    AND c.is_holiday = FALSE
),
weekly_grouped AS (
  SELECT
    fund_code,
    return_type,
    date_trunc('week', market_date) AS week_start_date,
    COUNT(daily_return) AS observed_trading_days,
    MIN(market_date) AS first_market_date_in_week,
    MAX(market_date) AS last_market_date_in_week,
    EXP(SUM(LN(1.0 + daily_return))) - 1.0 AS weekly_return
  FROM market_open_days
  WHERE daily_return IS NOT NULL
  GROUP BY 1,2,3
)
SELECT
  w.fund_code,
  w.return_type,
  w.week_start_date,
  e.expected_trading_days,
  w.observed_trading_days,
  ROUND(w.observed_trading_days::DOUBLE / NULLIF(e.expected_trading_days, 0), 4) AS coverage_ratio,
  (w.observed_trading_days = e.expected_trading_days) AS is_complete_week,
  w.first_market_date_in_week,
  w.last_market_date_in_week,
  w.weekly_return
FROM weekly_grouped w
INNER JOIN expected_days e
  ON e.week_start_date = w.week_start_date
ORDER BY w.fund_code, w.return_type, w.week_start_date
""").df()

weekly_returns_with_coverage_df
